In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
import requests

In [2]:
def download_file(url):
    file_path = url.split('/')[-1]
    resp = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(resp.content)
    return file_path

# 1. Redes de colaboração

- Artigos na área de astrofísica
- Publicados entre 1993 e 2003

[Link para o dataset](https://snap.stanford.edu/data/ca-AstroPh.html)

Referência:
- J. Leskovec, J. Kleinberg and C. Faloutsos. Graph Evolution: Densification and Shrinking Diameters. ACM Transactions on Knowledge Discovery from Data (ACM TKDD), 1(1), 2007.

In [3]:
url = 'https://snap.stanford.edu/data/ca-AstroPh.txt.gz'
file_path = download_file(url)
file_path

'ca-AstroPh.txt.gz'

In [4]:
G = nx.read_edgelist(file_path)
print(G)

Graph with 18772 nodes and 198110 edges


## Densidade da rede

In [5]:
nx.density(G)

0.0011244455715955115

## Conectividade

In [6]:
nx.is_connected(G)

False

Quantas **componentes conexas** existem na rede?

In [7]:
nx.number_connected_components(G)

290

In [8]:
components = nx.connected_components(G)
len(nx.connected_components(G))

TypeError: object of type 'generator' has no len()

Convertendo para `list` primeiro:

In [ ]:
components = nx.connected_components(G)
len(list(components))

In [ ]:
components = nx.connected_components(G)
sizes = []
for comp in components:
    sizes.append(len(comp))

Usando *list comprehension*:

In [ ]:
components = nx.connected_components(G)
sizes = [len(comp) for comp in components]

In [ ]:
sizes = pd.Series(sizes)
sizes.describe()

Vamos analisar apenas a **maior componente**. Esse é um processo bastante comum.

In [ ]:
components = nx.connected_components(G)
nodes = max(components, key=len)
nodes

In [ ]:
H = G.subgraph(nodes)
print(H)

Percentual de nós e arestas na **maior componente**:

In [ ]:
H.number_of_nodes() / G.number_of_nodes()

In [ ]:
H.number_of_edges() / G.number_of_edges()

## Distribuição dos graus

### Histograma

In [ ]:
degree_list = [degree for node, degree in H.degree]
degree_list = pd.Series(degree_list)
degree_list.hist();

In [ ]:
degree_list.describe()

### Gráfico de *ranqueamento*

Apesar de ter cauda longa, não parece seguir Lei de Potência

In [ ]:
y = degree_list.sort_values(ascending=False)
x = range(len(y))
plt.plot(x, y)
plt.loglog()
plt.xlabel('Posição (rank)')
plt.ylabel('Grau do nó');

## Distâncias

Cálculo pode ser bastante demorado. Por isso vamos usar o módulo `approximation`:

In [ ]:
nx.algorithms.approximation.diameter(H)

In [ ]:
top_nodes = [node for node, degree in H.degree if degree > 350]
top_nodes

In [ ]:
distances = nx.shortest_path_length(H, '1086')
distances = pd.Series(distances)
distances.describe()

In [ ]:
distances = nx.shortest_path_length(H, '62821')
distances = pd.Series(distances)
distances.describe()

## Clusterização

Mais uma vez, vamos usar uma **aproximação**:

In [ ]:
nx.algorithms.approximation.average_clustering(H)

## Comparação com rede aleatória Erdös-Rényi

In [ ]:
num_nodes = G.number_of_nodes()
density = nx.density(G)
random_graph = nx.erdos_renyi_graph(num_nodes, density, seed=42)
print(random_graph)

### Clusterização

In [ ]:
nx.algorithms.approximation.average_clustering(H)

In [ ]:
nx.algorithms.approximation.average_clustering(random_graph)

### Número de componentes

In [ ]:
nx.number_connected_components(G)

In [ ]:
nx.number_connected_components(random_graph)

### Diâmetro

In [ ]:
nx.algorithms.approximation.diameter(H)

In [ ]:
nx.algorithms.approximation.diameter(random_graph)

### Distribuição dos graus

In [ ]:
degree_list = [degree for node, degree in G.degree]
degree_list = pd.Series(degree_list)
degree_list.hist();

In [ ]:
rng_degree_list = [degree for node, degree in random_graph.degree]
rng_degree_list = pd.Series(rng_degree_list)
rng_degree_list.hist();

# 2. Internet

"*Fotografia*" tirada em novembro de 2007

[Link para o dataset](https://snap.stanford.edu/data/as-Caida.html)

Referência:
- J. Leskovec, J. Kleinberg and C. Faloutsos. Graphs over Time: Densification Laws, Shrinking Diameters and Possible Explanations. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2005.

In [ ]:
url = 'https://snap.stanford.edu/data/as-caida20071105.txt.gz'
file_path = download_file(url)
file_path

In [ ]:
G = nx.read_weighted_edgelist(file_path)
print(G)

## Densidade da rede

Dez vezes menor do que a rede de colaboração!

In [ ]:
nx.density(G)

## Conectividade

Faz sentido que a rede seja conectada:

In [ ]:
nx.is_connected(G)

Só existe uma **componente conexa** na rede:

In [ ]:
nx.number_connected_components(G)

## Distribuição dos graus

### Histograma

In [ ]:
degree_list = [degree for node, degree in G.degree]
degree_list = pd.Series(degree_list)
degree_list.hist();

In [ ]:
degree_list.describe()

### Gráfico de *ranqueamento*

Provavelmente uma Lei de Potência!

In [ ]:
y = degree_list.sort_values(ascending=False)
x = range(len(y))
plt.plot(x, y)
plt.loglog()
plt.xlabel('Posição (rank)')
plt.ylabel('Grau do nó');

## Distâncias

Novamente usaremos a aproximação:

In [ ]:
nx.algorithms.approximation.diameter(G)

In [ ]:
top_nodes = [node for node, degree in G.degree if degree > 1500]
top_nodes

In [ ]:
distances = nx.shortest_path_length(G, '174')
distances = pd.Series(distances)
distances.describe()

In [ ]:
distances = nx.shortest_path_length(G, '3356')
distances = pd.Series(distances)
distances.describe()

## Clusterização

Três vezes menor do que a da *Rede de colaboração*:

In [ ]:
nx.algorithms.approximation.average_clustering(G)

## Comparação com rede aleatória Erdös-Rényi

In [ ]:
num_nodes = G.number_of_nodes()
density = nx.density(G)
random_graph = nx.erdos_renyi_graph(num_nodes, density, seed=42)
print(random_graph)

### Clusterização

In [ ]:
nx.algorithms.approximation.average_clustering(G)

In [ ]:
nx.algorithms.approximation.average_clustering(random_graph)

### Número de componentes

In [ ]:
nx.number_connected_components(G)

In [ ]:
nx.number_connected_components(random_graph)

### Diâmetro

In [ ]:
nx.algorithms.approximation.diameter(G)

In [ ]:
nx.algorithms.approximation.diameter(random_graph)

Maior componente conexa do grafo aleatório:

In [ ]:
components = nx.connected_components(random_graph)
nodes = max(components, key=len)
random_graph_comp = random_graph.subgraph(nodes)
print(random_graph_comp)

In [ ]:
nx.algorithms.approximation.diameter(random_graph_comp)

### Distribuição dos graus

In [ ]:
degree_list = [degree for node, degree in G.degree]
degree_list = pd.Series(degree_list)
degree_list.hist();

In [ ]:
rng_degree_list = [degree for node, degree in random_graph.degree]
rng_degree_list = pd.Series(rng_degree_list)
rng_degree_list.hist();

# 3. Estradas

- Rede rodoviária do estado da Pensilvânia, EUA.
- Rede **direcionada**.

[Link para o dataset](https://snap.stanford.edu/data/roadNet-PA.html)

Referência:
- J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters. Internet Mathematics 6(1) 29--123, 2009.

In [ ]:
url = 'https://snap.stanford.edu/data/roadNet-PA.txt.gz'
file_path = download_file(url)
file_path

Essa rede pode demorar para carregar!

In [ ]:
G = nx.read_edgelist(file_path, create_using=nx.DiGraph)
print(G)

## Densidade da rede

Muito menor do que as outras! Por quê?

In [ ]:
nx.density(G)

## Conectividade

Fortemente conexo -> caminho de **ida e volta**

In [ ]:
nx.is_strongly_connected(G)

Quantas **componentes conexas fortes** existem na rede?

In [ ]:
nx.number_strongly_connected_components(G)

Separando a **maior componente conexa forte**:

In [ ]:
components = nx.strongly_connected_components(G)
nodes = max(components, key=len)
H = G.subgraph(nodes)
print(H)

Percentual de nós e arestas na **maior componente**:

In [ ]:
H.number_of_nodes() / G.number_of_nodes()

In [ ]:
H.number_of_edges() / G.number_of_edges()

## Distribuição dos graus

### Histograma

Grau de entrada:

In [ ]:
in_degree_list = [degree for node, degree in G.in_degree]
in_degree_list = pd.Series(in_degree_list)
in_degree_list.hist();

In [ ]:
in_degree_list.describe()

Grau de saída:

In [ ]:
out_degree_list = [degree for node, degree in G.out_degree]
out_degree_list = pd.Series(out_degree_list)
out_degree_list.hist();

In [ ]:
out_degree_list.describe()

### Gráfico de *ranqueamento*

Certamente não segue Lei de Potência:

In [ ]:
y = in_degree_list.sort_values(ascending=False)
x = range(len(y))
plt.plot(x, y)
plt.loglog()
plt.xlabel('Posição (rank)')
plt.ylabel('Grau do nó');

## Distâncias

Limite inferior para o diâmetro:

In [ ]:
nx.algorithms.approximation.diameter(H)

In [ ]:
distances = nx.shortest_path_length(H, '0')
distances = pd.Series(distances)
distances.describe()

## Clusterização

Vamos criar uma cópia **não direcionada** da rede:

In [ ]:
undirected = nx.to_undirected(H)
print(undirected)

Clusterização muito pequena! Faz sentido?

In [ ]:
nx.algorithms.approximation.average_clustering(undirected)

## Comparação com rede aleatória Erdös-Rényi

Rede muito grade -> **equações**

In [ ]:
n = G.number_of_nodes()
p = nx.density(G)
n, p

### Clusterização

Cada aresta tem probabilidade `p` de existir, a chance de haver uma aresta entre vizinhos é `p`

In [ ]:
random_graph_clustering = p
random_graph_clustering

### Número de componentes

Componente gigante surge se grau médio é **maior do que 1**!

In [ ]:
mean_degree = p * (n - 1)
mean_degree

### Diâmetro

Em uma rede aleatória, pode ser estimado pelo **número de nós** e **grau médio**:

In [ ]:
random_graph_diameter = np.log(n) / np.log(mean_degree)
random_graph_diameter

### Distribuição dos graus

Graus seguem uma **Distribuição Binomial**:

In [ ]:
degree_dist = scipy.stats.binom(n - 1, p)
x = np.arange(10)
y = degree_dist.pmf(x)
plt.plot(x, y)
plt.xlabel('Grau do nó')
plt.ylabel('Probabilidade');

In [ ]:
sns.displot(in_degree_list, stat='probability', bins=9)
plt.plot(x, y);

In [ ]:
rng_degree_list = [degree for node, degree in random_graph.degree]
rng_degree_list = pd.Series(rng_degree_list)
rng_degree_list.hist();

# 4. Redes sociais

- Rede social do site [Slashdot](https://slashdot.org/)
- Aresta *friend or foe*

[Link para o dataset](https://snap.stanford.edu/data/soc-Slashdot0811.html)

Referência:
- J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters. Internet Mathematics 6(1) 29--123, 2009.

In [ ]:
url = 'https://snap.stanford.edu/data/soc-Slashdot0811.txt.gz'
file_path = download_file(url)
file_path

In [ ]:
G = nx.read_edgelist(file_path, create_using=nx.DiGraph)
print(G)

## Densidade da rede

Muito parecida com a densidade da Internet!

In [ ]:
nx.density(G)

## Conectividade

Fortemente conexo -> caminho de **ida e volta**

In [ ]:
nx.is_strongly_connected(G)

Muitas **componentes conexas fortes**!

In [ ]:
nx.number_strongly_connected_components(G)

Separando a **maior componente conexa forte**:

In [ ]:
components = nx.strongly_connected_components(G)
nodes = max(components, key=len)
H = G.subgraph(nodes)
print(H)

Percentual de nós e arestas na **maior componente**:

In [ ]:
H.number_of_nodes() / G.number_of_nodes()

In [ ]:
H.number_of_edges() / G.number_of_edges()

Menor conectividade quando comparada às outras redes

## Distribuição dos graus

### Histograma

Grau de entrada:

In [ ]:
in_degree_list = [degree for node, degree in G.in_degree]
in_degree_list = pd.Series(in_degree_list)
in_degree_list.hist();

In [ ]:
in_degree_list.describe()

Grau de saída:

In [ ]:
out_degree_list = [degree for node, degree in G.out_degree]
out_degree_list = pd.Series(out_degree_list)
out_degree_list.hist();

In [ ]:
out_degree_list.describe()

Graus de entrada e saída são parecidos

### Gráfico de *ranqueamento*

Será que segue Lei de Potência?

In [ ]:
y = in_degree_list.sort_values(ascending=False)
x = range(len(y))
plt.plot(x, y)
plt.loglog()
plt.xlabel('Posição (rank)')
plt.ylabel('Grau do nó');

## Distâncias

Limite inferior para o diâmetro:

In [ ]:
nx.algorithms.approximation.diameter(H)

In [ ]:
distances = nx.shortest_path_length(H, '0')
distances = pd.Series(distances)
distances.describe()

## Clusterização

Vamos criar uma cópia **não direcionada** da rede:

In [ ]:
undirected = nx.to_undirected(H)
print(undirected)

Como em muitas redes sociais, clusterização é alta!

In [ ]:
nx.algorithms.approximation.average_clustering(undirected)

## Comparação com rede aleatória Erdös-Rényi

Vamos aproveitar as equações!

In [ ]:
n = G.number_of_nodes()
p = nx.density(G)
n, p

### Clusterização

Certamente não é uma rede aleatória!

In [ ]:
random_graph_clustering = p
random_graph_clustering

### Número de componentes

Com grau médio > 1, tendência de formar uma **componente gigante**

In [ ]:
mean_degree = p * (n - 1)
mean_degree

### Diâmetro

Na rede aleatória, esperamos um diâmetro menor:

In [ ]:
random_graph_diameter = np.log(n) / np.log(mean_degree)
random_graph_diameter

### Distribuição dos graus

Graus seguem uma **Distribuição Binomial**:

In [ ]:
degree_dist = scipy.stats.binom(n - 1, p)
x = np.arange(20)
y = degree_dist.pmf(x)
plt.plot(x, y)
plt.xlabel('Grau do nó')
plt.ylabel('Probabilidade');

In [ ]:
in_degree_list.hist();